In [9]:
import sqlite3
import numpy as np
import requests
import json
import logging
import os
from pypdf import PdfReader

# Logging configuration
logging.basicConfig(level=logging.INFO)

class Config:
    DB_NAME = "pdf_documents.db"
    TABLE_NAME = "pdf_documents"
    PAGES_TABLE = "pdf_pages"
    FILES_TABLE = "pdf_files"
    LOOKUP_TABLE = "pdf_lookup"
    EMBEDDING_MODEL_NAME = "mxbai-embed-large"
    OLLAMA_BASE_URL = "http://localhost:11434"
    VECTOR_DB = "pdf_vector.db"
    DIMS = 768  # Define vector dimensions

    @classmethod
    def load_from_file(cls, config_file="config.json"):
        if os.path.exists(config_file):
            try:
                with open(config_file, "r") as f:
                    config_data = json.load(f)
                for key, value in config_data.items():
                    if hasattr(cls, key):
                        setattr(cls, key, value)
                logging.info("Loaded configuration from file.")
            except Exception as e:
                logging.error(f"Failed to load configuration file: {e}")

# Load configuration from file if available
Config.load_from_file()

class PDFDatabase:
    def __init__(self, db_name=Config.DB_NAME):
        self.conn = sqlite3.connect(db_name)
        self.cursor = self.conn.cursor()
        self.create_tables()
    
    def create_tables(self):
        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {Config.TABLE_NAME} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                file_name TEXT UNIQUE,
                text TEXT
            )
        ''')
        
        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {Config.PAGES_TABLE} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                document_id INTEGER,
                page_number INTEGER,
                text TEXT,
                FOREIGN KEY(document_id) REFERENCES {Config.TABLE_NAME}(id)
            )
        ''')
        
        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {Config.FILES_TABLE} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                file_name TEXT UNIQUE,
                file BLOB
            )
        ''')
        
        self.cursor.execute(f'''
            CREATE TABLE IF NOT EXISTS {Config.LOOKUP_TABLE} (
                id INTEGER PRIMARY KEY,
                content TEXT
            )
        ''')
        self.conn.commit()
    
    def close(self):
        self.conn.close()

import sqlite_vec

class VectorDB:
    def __init__(self, db_name=Config.VECTOR_DB):
        self.conn = sqlite3.connect(db_name)
        self.conn.enable_load_extension(True)
        sqlite_vec.load(self.conn)
        self.conn.enable_load_extension(False)
        self.cursor = self.conn.cursor()
        self.create_tables()
    
    def create_tables(self):
        self.cursor.execute(f'CREATE VIRTUAL TABLE IF NOT EXISTS pdf_vec USING vec0(embedding float[{Config.DIMS}])')
        self.cursor.execute('CREATE TABLE IF NOT EXISTS pdf_lookup (id INTEGER PRIMARY KEY AUTOINCREMENT, content TEXT);')
        self.conn.commit()
    
    def search(self, fts_search_query: str, top_k: int = 2):
        fts_results = self.cursor.execute("SELECT id FROM pdf_fts WHERE pdf_fts MATCH ? ORDER BY rank LIMIT 5", (fts_search_query,)).fetchall()
        query_embedding = EmbeddingModel.generate_embeddings(fts_search_query)
        print(f"Query embedding: {query_embedding}")
        vec_results = self.cursor.execute("SELECT rowid, distance FROM pdf_vec WHERE embedding MATCH ? AND K = ? ORDER BY distance", [np.array(query_embedding, dtype=np.float32).tobytes(), top_k]).fetchall()
        
        combined_results = VectorDB.reciprocal_rank_fusion(fts_results, vec_results)
        for id, score in combined_results:
            print(f'ID: {id}, Content: {self.lookup_row(id)}, RRF Score: {score}')

    @staticmethod
    def serialize_f32(vec):
        return np.array(vec, dtype=np.float32).tobytes()

    @staticmethod
    def reciprocal_rank_fusion(fts_results, vec_results, k=60):
        rank_dict = {}

        # Process FTS results
        for rank, (id,) in enumerate(fts_results):
            if id not in rank_dict:
                rank_dict[id] = 0
            rank_dict[id] += 1 / (k + rank + 1)

        # Process vector results
        for rank, (rowid, distance) in enumerate(vec_results):
            if rowid not in rank_dict:
                rank_dict[rowid] = 0
            rank_dict[rowid] += 1 / (k + rank + 1)

        # Sort by RRF score
        sorted_results = sorted(rank_dict.items(), key=lambda x: x[1], reverse=True)
        return sorted_results

    @staticmethod
    def or_words(input_string):
        # Split the input string into words
        words = input_string.split()

        # Join the words with ' OR ' in between
        result = ' OR '.join(words)

        return result

    def lookup_row(self, id):
        row_lookup = self.cursor.execute('''
        SELECT content FROM pdf_lookup WHERE id = ?
        ''', (id,)).fetchall()
        content = ''
        for row in row_lookup:
            content= row[0]
            break
        return content


    def close(self):
        self.conn.close()

class EmbeddingModel:
    @staticmethod
    def generate_embeddings(text, model_name=Config.EMBEDDING_MODEL_NAME):
        try:
            print(f"Model: {model_name}")
            url = f"{Config.OLLAMA_BASE_URL}/api/embed"
            response = requests.post(url, json={"input": text, "model": model_name})
            response.raise_for_status()
            return response.json()["embeddings"]
        except requests.RequestException as e:
            logging.error(f"Embedding generation failed: {e}")
            return None

# Initialize database
pdf_db = PDFDatabase()
pdf_db.close()

# Initialize vector store
vector_db = VectorDB()
logging.info("Vector database initialized.")

logging.info("---- Technology ----")
vector_db.search("technology")
logging.info("---- Electric ----")
vector_db.search("Electric")
logging.info("---- Medical ----")
vector_db.search("medical")

vector_db.close()


INFO:root:Vector database initialized.
INFO:root:---- Technology ----


Model: mxbai-embed-large


INFO:root:---- Electric ----


Query embedding: [[-0.008968973, -0.0027457934, -0.012958789, 0.019653464, 0.006898165, 0.01787054, -0.0115903765, -0.017968966, 0.059480783, 0.05730865, -0.0128553435, 0.023200804, 0.04387384, -0.03382907, -0.0016747263, 0.005146793, -0.0044114995, -0.0058017494, -0.034737058, 0.012822302, -0.019806322, 0.030092012, -0.053189684, -0.03752967, -0.038981035, 0.02908877, -0.016704218, 0.01052948, 0.055150323, 0.0467547, -0.010662731, -0.016052036, 0.040281437, -0.036896404, -0.02632614, -0.01315118, -0.018535689, -0.0038567916, -0.025930814, -0.026304374, -0.008729751, -0.018153783, 0.024430541, -0.06793499, -0.053702556, 0.013021708, 0.022697488, -0.05802507, 0.007655279, 0.0039399243, -0.02217229, 0.0029505312, 0.014519167, -0.048644107, -0.0042473883, -0.017736424, -0.013886039, 0.023869535, -0.021528337, 0.033936635, 0.06251156, 0.011119075, 0.046752457, -0.028711498, 0.02957863, 0.025715956, 0.019076718, -0.023296319, -0.004022102, -0.00037977353, -0.038017076, 0.017865073, -0.02706

INFO:root:---- Medical ----


Query embedding: [[-0.038288124, -0.003532887, -0.036974166, -0.0065555405, -0.041167993, -0.013605245, -0.0052484404, -0.013532711, 0.04056231, 0.026345838, 0.015770791, 0.0043054116, 0.0350687, -0.046726964, -0.028780624, 0.0010349187, -0.06831596, -0.011117923, -0.052548625, 0.03876141, 0.012211253, 0.01730825, -0.018630276, -0.03777927, -0.03516263, 0.014774833, -0.015626188, -0.01085214, 0.030584102, 0.03888023, -0.013240754, -0.011212674, 0.023844205, -0.012456059, -0.015449745, -0.026844831, -0.026268976, -0.030398954, -0.034616463, -0.0684481, 0.00637884, -0.021840626, 0.010991724, -0.0019943682, -0.029436054, -0.017718172, -0.016867824, -0.04572496, 0.016474562, -0.063323036, -0.015950965, 0.013141817, 0.003952307, 0.019535255, 0.010461764, -0.009958137, -0.032068882, -0.0031665107, -0.041630663, 0.037212398, 0.041346997, -0.020792488, 0.053984337, -0.02059285, -0.032396927, 0.023685956, 0.0076306826, -0.0052206907, 0.027824314, 0.012680422, -0.00083297264, 0.014138893, -0.030

In [10]:
from smolagents import Tool

class RetrieverTool(Tool):
    name = "retriever"
    description = (
        "Uses semantic search to retrieve the parts of documentation that could be most relevant to answer your query."
    )
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vector_store, **kwargs):
        super().__init__(**kwargs)
        self.vector_store = vector_store

    def forward(self, query: str) -> str:
        docs = self.vector_store.search(query, k=3)
        return "\nRetrieved documents:\n" + "".join(
            [f"\n\n===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

vector_store = VectorDB()
retriever_tool = RetrieverTool(vector_store=vector_store)



INFO:numexpr.utils:Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO:numexpr.utils:NumExpr defaulting to 16 threads.
INFO:httpx:HTTP Request: GET https://raw.githubusercontent.com/BerriAI/litellm/main/model_prices_and_context_window.json "HTTP/1.1 200 OK"


In [11]:
from smolagents import LiteLLMModel, CodeAgent

model = LiteLLMModel(
    model_id="ollama_chat/qwen2.5",
    api_base="http://localhost:11434/",
)

agent = CodeAgent(
    tools=[retriever_tool],
    model=model,
    max_steps=4,
    verbosity_level=2,
)

agent_output = agent.run("How can I push a model to the Hub?")


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How can I push a model to the Hub?                                                                              │
│                                                                                                                 │
╰─ LiteLLMModel - ollama_chat/qwen2.5 ────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

14:54:28 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= qwen2.5; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= qwen2.5; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434//api/chat "HTTP/1.1 301 Moved Permanently"



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Error in generating model output:
litellm.APIConnectionError: Ollama_chatException - Redirect response '301 Moved Permanently' for url 
'http://localhost:11434//api/chat'
Redirect location: '/api/chat'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/301

...

[Step 3: Duration 0.02 seconds]

14:54:31 - LiteLLM:INFO: utils.py:2905 - 
LiteLLM completion() model= qwen2.5; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= qwen2.5; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434//api/chat "HTTP/1.1 301 Moved Permanently"



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm._turn_on_debug()'.



Reached max steps.

[Step 4: Duration 0.02 seconds]